In [1]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset_25' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'HR_data')
output_dir = os.path.join(home, 'video_resources', 'sql_database')

In [2]:
from sql_face.alchemy import get_session
session = get_session(output_dir, db_name)

2023-06-28 00:17:31.411829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import  QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup,Detector, EmbeddingModel,EnfsiImage,EnfsiVideoFrame

Ficheros a importar

In [4]:
database = 'enfsi'

emb_file = os.path.join(input_dir,f'embeddings_{database}_hr.npy')
filenames_file = os.path.join(input_dir,f'filenames_{database}_hr.npy')
quality_file = os.path.join(input_dir,f'results_{database}_hr.csv')

emb = np.load(emb_file) 
filename = np.load(filenames_file)
df_quality = pd.read_csv(quality_file)


In [5]:
df_quality

,Path,Quality
0,/content/CodeFormer/align_hr_enfsi/28052.png,0.784107
1,/content/CodeFormer/align_hr_enfsi/29371.png,0.626369
2,/content/CodeFormer/align_hr_enfsi/26453.png,0.468202
3,/content/CodeFormer/align_hr_enfsi/29537.png,0.862583
4,/content/CodeFormer/align_hr_enfsi/30206.png,0.819336
...,...,...
2471,/content/CodeFormer/align_hr_enfsi/28719.png,0.000782
2472,/content/CodeFormer/align_hr_enfsi/30022.png,0.028895
2473,/content/CodeFormer/align_hr_enfsi/26889.png,0.000379
2474,/content/CodeFormer/align_hr_enfsi/27171.png,0.051150


In [6]:
print(filename.shape,emb.shape, df_quality.shape)


(2476,) (2476, 512) (2476, 2)


In [7]:
df = pd.DataFrame({'Path': filename, 'embedding': emb.tolist()})
df

,Path,embedding
0,/content/CodeFormer/align_hr_enfsi2015/28052.png,"[0.14251266419887543, 1.6241755485534668, -1.0..."
1,/content/CodeFormer/align_hr_enfsi2015/29371.png,"[-0.9632573127746582, -0.06028220057487488, 1...."
2,/content/CodeFormer/align_hr_enfsi2015/26453.png,"[0.4431183338165283, -0.1789988875389099, 0.47..."
3,/content/CodeFormer/align_hr_enfsi2015/29537.png,"[-2.2235913276672363, 0.5421461462974548, -1.4..."
4,/content/CodeFormer/align_hr_enfsi2015/30206.png,"[0.2170746922492981, 0.5398297905921936, 0.714..."
...,...,...
2471,/content/CodeFormer/align_hr_enfsi2015/28719.png,"[-0.04542823135852814, -0.5117424726486206, 0...."
2472,/content/CodeFormer/align_hr_enfsi2015/30022.png,"[-0.6927948594093323, 0.07162030786275864, 1.3..."
2473,/content/CodeFormer/align_hr_enfsi2015/26889.png,"[0.8128702044487, 1.0840398073196411, -0.05720..."
2474,/content/CodeFormer/align_hr_enfsi2015/27171.png,"[1.7465705871582031, -0.6921619176864624, 1.02..."


In [8]:
df['CI_id'] = df.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))
df_quality['CI_id'] = df_quality.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))

In [9]:
results_data = df.merge(df_quality, on='CI_id')

In [10]:
results_data.shape

(2476, 5)

In [11]:
results_data

,Path_x,embedding,CI_id,Path_y,Quality
0,/content/CodeFormer/align_hr_enfsi2015/28052.png,"[0.14251266419887543, 1.6241755485534668, -1.0...",28052,/content/CodeFormer/align_hr_enfsi/28052.png,0.784107
1,/content/CodeFormer/align_hr_enfsi2015/29371.png,"[-0.9632573127746582, -0.06028220057487488, 1....",29371,/content/CodeFormer/align_hr_enfsi/29371.png,0.626369
2,/content/CodeFormer/align_hr_enfsi2015/26453.png,"[0.4431183338165283, -0.1789988875389099, 0.47...",26453,/content/CodeFormer/align_hr_enfsi/26453.png,0.468202
3,/content/CodeFormer/align_hr_enfsi2015/29537.png,"[-2.2235913276672363, 0.5421461462974548, -1.4...",29537,/content/CodeFormer/align_hr_enfsi/29537.png,0.862583
4,/content/CodeFormer/align_hr_enfsi2015/30206.png,"[0.2170746922492981, 0.5398297905921936, 0.714...",30206,/content/CodeFormer/align_hr_enfsi/30206.png,0.819336
...,...,...,...,...,...
2471,/content/CodeFormer/align_hr_enfsi2015/28719.png,"[-0.04542823135852814, -0.5117424726486206, 0....",28719,/content/CodeFormer/align_hr_enfsi/28719.png,0.000782
2472,/content/CodeFormer/align_hr_enfsi2015/30022.png,"[-0.6927948594093323, 0.07162030786275864, 1.3...",30022,/content/CodeFormer/align_hr_enfsi/30022.png,0.028895
2473,/content/CodeFormer/align_hr_enfsi2015/26889.png,"[0.8128702044487, 1.0840398073196411, -0.05720...",26889,/content/CodeFormer/align_hr_enfsi/26889.png,0.000379
2474,/content/CodeFormer/align_hr_enfsi2015/27171.png,"[1.7465705871582031, -0.6921619176864624, 1.02...",27171,/content/CodeFormer/align_hr_enfsi/27171.png,0.051150


In [12]:
results_data.drop(['Path_x','Path_y'] , axis=1, inplace = True)

In [13]:
results_data

,embedding,CI_id,Quality
0,"[0.14251266419887543, 1.6241755485534668, -1.0...",28052,0.784107
1,"[-0.9632573127746582, -0.06028220057487488, 1....",29371,0.626369
2,"[0.4431183338165283, -0.1789988875389099, 0.47...",26453,0.468202
3,"[-2.2235913276672363, 0.5421461462974548, -1.4...",29537,0.862583
4,"[0.2170746922492981, 0.5398297905921936, 0.714...",30206,0.819336
...,...,...,...
2471,"[-0.04542823135852814, -0.5117424726486206, 0....",28719,0.000782
2472,"[-0.6927948594093323, 0.07162030786275864, 1.3...",30022,0.028895
2473,"[0.8128702044487, 1.0840398073196411, -0.05720...",26889,0.000379
2474,"[1.7465705871582031, -0.6921619176864624, 1.02...",27171,0.051150


# Datos para la base de datos
Revisar estos datos si cambiamos de database. o hacer las queries para obtenerlos. 

In [14]:
detector_id = 2 # mtcnn-serfiq
embeddingModel_id = 6 #El nuevo
qualityModel_id = 1 #ser_fiq

# Creamos/Actualizamos FaceImage

In [15]:
#buscamos la faceimage y la creamos si no existe
limit = 300
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    #cI = session.query(CroppedImage).join(Image).filter(Image.path == dato.Path,CroppedImage.detector_id==detector_id).one_or_none()
    cI = session.query(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id,CroppedImage.detector_id==detector_id).one_or_none()

    if cI:
        FI = session.query(FaceImage).filter(FaceImage.croppedImages == cI,FaceImage.embeddingModel_id == embeddingModel_id).one_or_none()
        if FI:
            FI.embeddings=dato.embedding

        else:
            new_face_image = FaceImage(croppedImage_id=cI.croppedImage_id,
            embeddingModel_id=embeddingModel_id,
            embeddings=dato.embedding,
            confusion_score=None)
            session.add(new_face_image)
        i-=1
        #session.commit()        

    else:
        print (f'{dato.Path} no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()



0it [00:00, ?it/s]

2476it [00:12, 194.29it/s]


# Creamos / actualizamos QualityImage

In [16]:
#buscamos la qualityimage y la creamos si no existe
limit = 100
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    FI = session.query(FaceImage).filter(FaceImage.embeddingModel_id == embeddingModel_id) \
                .join(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id).one_or_none()
#                .join(Image).filter(Image.path == dato.Path)

    if FI:
        QI = session.query(QualityImage).filter(QualityImage.faceImages == FI, QualityImage.qualityModel_id == qualityModel_id).one_or_none()
        if QI:
            QI.quality=dato.Quality

        else:
            new_quality_image = QualityImage(faceImage_id = FI.faceImage_id,
            qualityModel_id = qualityModel_id,
            quality=dato.Quality)
            session.add(new_quality_image)
        i-=1
        #session.commit()        

    else:
        print (dato.Path + 'no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()





0it [00:00, ?it/s]

2476it [00:13, 178.05it/s]
